# Figure 5

Final version of Figure 5 for Hogg et al. (2022)

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.path as mpath
import xarray as xr
import numpy as np
import datetime as dt
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
from scipy import stats
from dask.distributed import Client

In [4]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 44.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45209,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 44.92 GiB
Comm: tcp://127.0.0.1:43463,Total threads: 4
Dashboard: /proxy/44351/status,Memory: 11.23 GiB
Nanny: tcp://127.0.0.1:33507,


In [5]:
HardDisk = '/g/data/v45/amh157/OCCIPUT/'
EKEDir = 'processed_eke/'

In [6]:
land_50m = cft.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='black',
                                        facecolor='gray',
                                        linewidth=1.0)

# Load wind stress File and coarsen by a factor nn

In [7]:
# Load tau and coarsen by factor nn
nn=16
taufilename = HardDisk+'processed_tau/'+'tau_????.nc'
tau = xr.open_mfdataset(taufilename).tau.sel(lat=slice(-80,-30))
tau_coarse = tau.coarsen({"lon":nn,"lat":nn},boundary='pad').mean().load()
tau_coarse = tau_coarse.rename(tau.name+'_coarse'+str(nn))

tau_clim = tau_coarse.groupby('time.month').mean('time')
tau_deseas = (tau_coarse.groupby('time.month') - tau_clim).load()

# Function to compute map of correlation coefficient at specified lag

In [8]:
def corrmap(jj, tau_cp, ekep):
    x1t = np.roll(tau_cp, jj,0)[jj:]
    x2t = ekep[jj:]

    nx = tau_cp.lon.size
    ny = tau_cp.lat.size
    corr_mat = np.zeros([ny,nx])
    for i in range(nx):
        for j in range(ny):
            if ~np.isnan(x1t[0,j,i]):
                corr_mat[j,i], sig = stats.pearsonr(x1t[:,j,i], x2t)

    return corr_mat

## Compute variance ratios from saved results

In [9]:
def CalcThisBox(latlon):

    ## Other things to set
    nfiles = 50
    nn=0
    start = dt.datetime(1980,1,1) ## start time for plotting
    end = dt.datetime(2014,12,31)

    ## plot EKE
    eke_results = []
    for ii in range(1,nfiles+1):
        nn+=1
        i00 = '%03d' % ii
        Member = 'e'+i00
        ekefilename = HardDisk+EKEDir+'/eke_'+i00+'_????.nc'
        
        eke = xr.open_mfdataset(ekefilename).EKE
    
        eke = eke.sel(lon=slice(latlon['lon1'],latlon['lon2'])).sel(lat=slice(latlon['lat1'],latlon['lat2'])).mean('lon').mean('lat')
        eke_clim = eke.groupby('time.month').mean('time')
        eke_deseas = (eke.groupby('time.month') - eke_clim).load()
        #eke = eke.rolling(time=12,center=True).mean().load()    
        result = {'ii': ii,'eke': eke_deseas}
        eke_results.append(result)
    
        if ii==1:
            eke_sum = eke
        else:
            eke_sum = eke_sum + eke
        #print(ekefilename)
                        
    eke_sum = eke_sum.load()/nn
    eke_sum_clim = eke_sum.groupby('time.month').mean('time')
    eke_sum_deseas = (eke_sum.groupby('time.month') - eke_sum_clim).load()
    #ekep = eke_sum[6:-6] - eke_sum.mean()
    
    return eke_sum_deseas

## Now calculate correlation maps for each region

In [10]:
##
SEIO = {'lon1': -285, 'lon2': -240, 'lat1': -60, 'lat2':-40}
eke_deseas = CalcThisBox(SEIO)
corr_mat_SEIO_5 = corrmap(5, tau_deseas, eke_deseas)
corr_mat_SEIO_30 = corrmap(30, tau_deseas, eke_deseas)

In [11]:
SWPO = {'lon1': -215, 'lon2': -160, 'lat1': -60, 'lat2':-42}
eke_deseas = CalcThisBox(SWPO)
corr_mat_SWPO_15 = corrmap(15, tau_deseas, eke_deseas)

In [12]:
SAO = {'lon1': -40, 'lon2': 10, 'lat1': -55, 'lat2':-40}
eke_deseas = CalcThisBox(SAO)
corr_mat_SAO_17 = corrmap(17, tau_deseas, eke_deseas)

CPU times: user 28.3 s, sys: 3.74 s, total: 32 s
Wall time: 29 s


In [13]:
%%time
plt.figure(figsize=(10, 10))

ax1 = plt.subplot(2,2,1,projection=ccrs.SouthPolarStereo())
ax2 = plt.subplot(2,2,2,projection=ccrs.SouthPolarStereo())
ax3 = plt.subplot(2,2,3,projection=ccrs.SouthPolarStereo())
ax4 = plt.subplot(2,2,4,projection=ccrs.SouthPolarStereo())

# Compute a circle in axes coordinates, which we can use as a boundary
# for the map. We can pan/zoom as much as we like - the boundary will be
# permanently circular
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax1.set_extent([-287, 73, -80, -35], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)
ax1.add_feature(land_50m)
ax1.gridlines()
p1=ax1.pcolor(tau_deseas.lon, tau_deseas.lat, corr_mat_SEIO_5, vmin=-0.5, vmax=0.5,cmap=cm.cm.tarn_r,transform=ccrs.PlateCarree())
lon1=SEIO['lon1']
lon2=SEIO['lon2']
lat1=SEIO['lat1']
lat2=SEIO['lat2']
ax1.plot([lon1, lon1],[lat1,lat2], color='c',linewidth=2,transform=ccrs.PlateCarree())
ax1.plot([lon2, lon2],[lat1,lat2], color='c',linewidth=2,transform=ccrs.PlateCarree())
ax1.plot(np.arange(lon1,lon2+1),lat1*np.ones(lon2+1-lon1),
            color='c',linewidth=2,transform=ccrs.PlateCarree())
ax1.plot(np.arange(lon1,lon2+1),lat2*np.ones(lon2+1-lon1),
            color='c',linewidth=2,transform=ccrs.PlateCarree())
ax1.set_title('(a) Southeast Indian; 5-month lag')

ax2.set_extent([-287, 73, -80, -35], crs=ccrs.PlateCarree())
ax2.set_boundary(circle, transform=ax2.transAxes)
ax2.add_feature(land_50m)
ax2.gridlines()
p2=ax2.pcolor(tau_deseas.lon, tau_deseas.lat, corr_mat_SEIO_30, vmin=-0.5, vmax=0.5,cmap=cm.cm.tarn_r,transform=ccrs.PlateCarree())
ax2.plot([lon1, lon1],[lat1,lat2], color='c',linewidth=2,transform=ccrs.PlateCarree())
ax2.plot([lon2, lon2],[lat1,lat2], color='c',linewidth=2,transform=ccrs.PlateCarree())
ax2.plot(np.arange(lon1,lon2+1),lat1*np.ones(lon2+1-lon1),
            color='c',linewidth=2,transform=ccrs.PlateCarree())
ax2.plot(np.arange(lon1,lon2+1),lat2*np.ones(lon2+1-lon1),
            color='c',linewidth=2,transform=ccrs.PlateCarree())
ax2.set_title('(b) Southeast Indian; 30-month lag')

ax3.set_extent([-287, 73, -80, -35], crs=ccrs.PlateCarree())
ax3.set_boundary(circle, transform=ax3.transAxes)
ax3.add_feature(land_50m)
ax3.gridlines()
p3=ax3.pcolor(tau_deseas.lon, tau_deseas.lat, corr_mat_SWPO_15, vmin=-0.5, vmax=0.5,cmap=cm.cm.tarn_r,transform=ccrs.PlateCarree())
lon1=SWPO['lon1']
lon2=SWPO['lon2']
lat1=SWPO['lat1']
lat2=SWPO['lat2']
ax3.plot([lon1, lon1],[lat1,lat2], color='r',linewidth=2,transform=ccrs.PlateCarree())
ax3.plot([lon2, lon2],[lat1,lat2], color='r',linewidth=2,transform=ccrs.PlateCarree())
ax3.plot(np.arange(lon1,lon2+1),lat1*np.ones(lon2+1-lon1),
            color='r',linewidth=2,transform=ccrs.PlateCarree())
ax3.plot(np.arange(lon1,lon2+1),lat2*np.ones(lon2+1-lon1),
            color='r',linewidth=2,transform=ccrs.PlateCarree())
ax3.set_title('(c) Southwest Pacific; 15-month lag')

ax4.set_extent([-287, 73, -80, -35], crs=ccrs.PlateCarree())
ax4.set_boundary(circle, transform=ax4.transAxes)
ax4.add_feature(land_50m)
ax4.gridlines()
p4=ax4.pcolor(tau_deseas.lon, tau_deseas.lat, corr_mat_SAO_17, vmin=-0.5, vmax=0.5,cmap=cm.cm.tarn_r,transform=ccrs.PlateCarree())
ax4.plot([SAO['lon1'], SAO['lon2'], SAO['lon2'], SAO['lon1'], SAO['lon1']],
          [SAO['lat1'],SAO['lat1'],SAO['lat2'],SAO['lat2'],SAO['lat1']],
         color='darkorange',linewidth=2,transform=ccrs.PlateCarree())
lon1=SAO['lon1']
lon2=SAO['lon2']
lat1=SAO['lat1']
lat2=SAO['lat2']
ax4.plot([lon1, lon1],[lat1,lat2], color='darkorange',linewidth=2,transform=ccrs.PlateCarree())
ax4.plot([lon2, lon2],[lat1,lat2], color='darkorange',linewidth=2,transform=ccrs.PlateCarree())
ax4.plot(np.arange(lon1,lon2+1),lat1*np.ones(lon2+1-lon1),
            color='darkorange',linewidth=2,transform=ccrs.PlateCarree())
ax4.plot(np.arange(lon1,lon2+1),lat2*np.ones(lon2+1-lon1),
            color='darkorange',linewidth=2,transform=ccrs.PlateCarree())
ax4.set_title('(d) South Atlantic; 17-month lag')

plt.subplots_adjust(left=0.05, bottom=0.05, top=0.95, right = 0.9)

ax_c = plt.axes([0.92,0.4,0.01,0.3])
plt.colorbar(p1,cax=ax_c,orientation='vertical',extend='both')

plt.savefig('/g/data/x77/amh157/OCCIPUT-analysis/revised_paper/figures/Figure5.png',dpi=200)